In [1]:
import youtube_analysis
from mysql.connector import Error
import mysql.connector
from flask import Flask, render_template, request, redirect, session
import sys
import logging
import secrets
import cohere
import random
import pandas as pd

In [2]:
def connect():
    try:
        connection = mysql.connector.connect(
            host='35.193.136.209',
            database='YoutubeTrending',
            user='root',
            password='test1234')
        if connection.is_connected():
            print('Connected to MySQL database')
            return connection
    except Error as e:
        print(f'Error connecting to MySQL database: {e}')

In [4]:
connection = connect()
cursor = connection.cursor()

# Query to get the totalPublished, totalLiked, totalChannels, and totalViews for each category
query = "SELECT categoryId, COUNT(video_id) AS totalPublished, SUM(likes) AS totalLiked, COUNT(DISTINCT channelId) AS totalChannels, SUM(view_count) AS totalViews FROM VideoInfo GROUP BY categoryId"

cursor.execute(query)
results = cursor.fetchall()
results

Connected to MySQL database


[(0, 1, Decimal('0'), 1, Decimal('0')),
 (1, 1305, Decimal('137191815'), 307, Decimal('3492973405')),
 (2, 647, Decimal('29853986'), 143, Decimal('725037251')),
 (10, 5426, Decimal('1530857650'), 1447, Decimal('26963943278')),
 (15, 150, Decimal('9610088'), 38, Decimal('221566198')),
 (17, 4250, Decimal('165943687'), 546, Decimal('7406898205')),
 (19, 184, Decimal('12009249'), 44, Decimal('241735135')),
 (20, 6791, Decimal('637651216'), 1428, Decimal('12799022938')),
 (22, 3065, Decimal('334938760'), 924, Decimal('6145286489')),
 (23, 1765, Decimal('227436414'), 347, Decimal('3275311344')),
 (24, 6752, Decimal('952467175'), 1471, Decimal('20384031591')),
 (25, 1312, Decimal('27565971'), 305, Decimal('2021003442')),
 (26, 923, Decimal('69527661'), 210, Decimal('1439144350')),
 (27, 807, Decimal('89638566'), 172, Decimal('1711676557')),
 (28, 1047, Decimal('112126830'), 190, Decimal('3089281549')),
 (29, 18, Decimal('3117567'), 13, Decimal('38611288'))]

In [5]:
# Loop through the results and insert/update the data in the CategoryInfo table
for result in results:
    categoryId = result[0]
    totalPublished = result[1]
    totalLiked = result[2]
    totalChannels = result[3]
    totalViews = result[4]

    # Check if the category already exists in the CategoryInfo table
    query = "SELECT * FROM CategoryInfo WHERE categoryId = %s"
    values = (categoryId,)
    cursor.execute(query, values)
    category = cursor.fetchone()

    if category:
        # If the category exists, update the data
        query = "UPDATE CategoryInfo SET totalPublished = %s, totalLiked = %s, totalChannels = %s, totalViews = %s WHERE categoryId = %s"
        values = (totalPublished, totalLiked, totalChannels, totalViews, categoryId)
        cursor.execute(query, values)
    else:
        # If the category does not exist, insert the data
        query = "INSERT INTO CategoryInfo (categoryId, totalPublished, totalLiked, totalChannels, totalViews) VALUES (%s, %s, %s, %s, %s)"
        values = (categoryId, totalPublished, totalLiked, totalChannels, totalViews)
        cursor.execute(query, values)

connection.commit()
# Close the database connection
connection.close()